<a href="https://colab.research.google.com/github/svvsaga/datascience_workshop/blob/martin/workshop_sesjon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Outline
Her tar vi utgangspunkt i GPS-data fra Peder sin kjøretur fra Trondheim til Ørsta

1. Laste opp fil til GCS via terminalen (Cloud Shell/gsutil)
  - Nevne at det også kan gjøres via GUI og client libs
2. Importere data direkte fra GCS til BigQuery (Cloud Shell/bq)
  - Nevne at det også kan gjøres via GUI og client libs
3. Utføre en enkel BigQuery-spørring for å se at vi har fått inn data
4. Legge til datamapping i en Dataflow-jobb, og importere og mappe data inn i BigQuery
  - Kan bruke samme data som i pkt. 2, men at vi i DF-jobben forbedrer datamodellen, f.eks. ved å slå sammen to kolonner lat, long til en geography-kolonne
  - Bør vise Dataflow-GUI mens jobben kjører
5. Sjekke i BigQuery at dataene har komt inn på riktig format
6. Vise at dei kan laste inn alle dataene frå GCS til BigQuery via GUI
7. Meir? Skal vi joine med geolokasjon i denne bolken eller seinare?
  - https://www.vegvesen.no/trafikkdata/api/?query=%7B%0A%20%20trafficRegistrationPoints%20%7B%0A%20%20%20%20id%0A%20%20%20%20name%0A%20%20%20%20location%20%7B%0A%20%20%20%20%20%20coordinates%20%7B%0A%20%20%20%20%20%20%20%20latLon%20%7B%0A%20%20%20%20%20%20%20%20%20%20lat%0A%20%20%20%20%20%20%20%20%20%20lon%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A

## Andre forslag:
- Bruke python libs + dataframes + pandas til noe (sesjon 1 eller 2?)
- Vurdere å vise Cloud Function som henter data og dytter det inn i Dataflow eller BigQuery?
- Andre verktøy: AirByte, FME og DBT

## Nytt forslag:
- Ta utgangspunkt i trafikkdata (saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest)
- Gjere cirka det samme som før i steg 1, 2, 3
- Då ser vi at dataene for det første er veldig nøsta og at vi ikkje veit "ingest time" som er ein del av filnamna.
- Steg 4 (dataflow) vil då kunne gi verdi pga:
  1. Kan ha med ingesttime inn i BigQuery
  2. Kan hente ut berre totalt trafikkvolum, sidan vi ikkje bryr oss om resten

TODO: Forklare at vi skal autentisere oss

---




### 1. Laste opp fil til GCS 
Det finnes flere måter å laster data inn til google cloud storage med ulike nivåer av kode-bruk:
- gjennom google cloud platform grafisk user interface (nettleser)
- gjennom google cloud shell (nettleser)
- gjennom lokal installasjon av google cloud SDK
- ved bruk av client bliblioteker (f.eks. python client)

Her vil vi introdusere muligheter ved bruk av cloud shell og lokal installasjon av cloud SDK. 

Kommandoene er i utgangspunkt de samme, men oppsett er litt forskjellig: 




#### Oppsett Cloud shell
- Logg deg inn på Google Cloud Platform UI gjennom nettleseren: https://console.cloud.google.com/ 
- Verifiser at prosjektet til workshop et valgt som arbeidsprosjekt
- Aktiver cloud shell ("Activate cloud shell" knapp i øverste høyre hjørnet i UI), vent til cloud shell er provisjonert


#### Oppsett Google cloud SDK
- Last ned og installer Google cloud SDK for ditt system (i forkant av workshoppen): https://cloud.google.com/sdk/docs/install 
- initialiser cloud shell (verifiser at prosjektet er satt til ditt arbeids/workshop prosjekt)



#### Last ned eksempelfiler fra GCS (gjennom UI)
- Last ned 2 filer fra
 `gs://saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest`. Dette er en bøtte som finnes i prosjektet `saga-trafikkdata-prod-pz8l` og som dere har fått rettigheter til å laste ned filer fra.
- Last ned de to filene:
 - `2021-03-23T09:31:31.945Z_2210582261344715.ndjson`
 - `2021-03-23T09:31:42.657Z_2210582990408255.ndjson`
- Dette kan oppnås ved å bruke følgende cloud/SDK shell kommando:

```
gsutil cp gs://bucket-name path/to/file.csv
```

#### Lag en GCS bøtte
Lag en GCS bøtte der filene skal lastest opp. 
Dette kan gjøres enten gjennom grafisk user interface eller gjennom cloud shell/SDK

- I grafisk user interface naviger til Cloud Storage -> Browser og trykk på "Create bucket" 
- Velg et (globalt unik!) navn til bucket og konfigurer lagringsopsjoner
  - standard storage
  - single-region, europe-west3
  - uniform access

Det samme kan oppnås ved bruk av følgende cloud/SDK shell kommando:

```
gsutil mb -b -c standard -l EUROPE-WEST3 gs://bucket-name
```

#### Laste opp filer til GCS bucket 
Laste opp lokale filer til GCS bucket ved bruk av cloud/SDK shell.

```
gsutil cp path/to/file.ndjson gs://bucket-name
```

Det er mulig å laste opp enkelte filter eller en liste med filter samtidig

```
gsutil cp path/to/*.ndjson gs://bucket-name
```

Til slutt verifiser at filene har blitt lastet opp, enten ved grafisk user interace (storage browser) eller gjennom cloud shell.

```
gsutil ls gs://bucket-name
```

En oversikt over gsutil kommandos finnes her: https://cloud.google.com/storage/docs/gsutil eller ved bruk av `gsutil help` i cloud shell terminal.

#### Laste opp filer med klientbiblioteker.
- Filer kan også lastes opp ved bruk av client bibliotheker. En oversikt over støttede programmeringsspråk finnes her: https://cloud.google.com/storage/docs/reference/libraries 

- I cellen under finnes det et minimalt eksempel med python client bibliothek.

- For å kjøre python kode direkte fra denne notebook, må google cloud brukeren først autentiseres. 

- Vi bruker google.colab.auth python bibliothek til autentisering og google.cloud.storage til interaksjon med storage buckets

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated') 

In [ ]:
from google.cloud import storage
from google.cloud import bigquery
%load_ext google.colab.data_table

proj = # TODO define workshop-project
bucket =  # TODO define gs://bucket-name
file_name = # TODO define lokal file name
blob_name = # TODO define GCS blob name for the file
client = storage.Client(project = proj) # initialize client and set the billing project. NB: vi trenger ikke å gjenta credentials her.
bucket_object = client.get_bucket(bucket) # define the target bucket
blob = bucket.blob(blob_name) # make a file blob
blob.upload_from_filename(file_name) # upload the content from lokal file to GCS

NB: Det finnes klientbiblioteker til de fleste GCP verktøy/tjenester. F.eks i python: https://cloud.google.com/python/docs/reference 

### 2. Importere data direkte fra GCS til BigQuery
Etter dataene har blitt lastet opp til GCS skal de ofte importeres til BigQuery til videre analyse. BigQuery er et datavarehus som kan lagre store datasett hvor man kan bruke SQL spørringer for å hente ut data.

BigQuery støtter en del formater direkte: 
- Avro
- CSV
- JSON
- ORC
- Parquet

Dataene kan importeres fra GCS bøtter, lokale filer (max 10MB upload per fil), google drive, BigTable eller genereres on-the-fly (lage en tom tabell).


Her importerer vi dataene vi har lastet opp til GCS bøtte til BigQuery ved bruk av cloud shell kommando. BigQuery organiserer dataene i prosjekter, datasett og så tabeller. Før vi kan laste opp dataene i en tabell må vi derfor lage en ny datasett: (flytt denne til når vi skal kjøre dataflow jobben, siden det allerede finnes et raw dataset som er lagd med terraform?)

```
bq --location=EUROPE-WEST3 mk -d dataset_name
```

Når datasett har blitt opprettet kan vi importere dataene fra GCS inn til en BigQuery tabell under samme datasett.


```
bq --location=europe-west3 load --autodetect --source_format=NEWLINE_DELIMITED_JSON DATASET.TABLE_ID gs://path-to-file
```

***
Do we want to load all trips into one table? Use e.g. file{1..3}.ndjson to upload more than one to its one file in one command
***

```
bq --location=europe-west3 load --autodetect --source_format=NEWLINE_DELIMITED_JSON DATASET.TABLE_ID gs://path-to-data-folder/2021-04-14*.ndjson
```

#### Verifser import med en SQL spørring

Før vi kan hente ut data fra BigQuery via SQL spørringer så må vi definere prosjekt, dataset og tabel IDene.

In [ ]:
project_id = "<prosjekt navn fra GCP>"
dataset_id = "raw"
table_id = "<table id>"

In [ ]:
project_id = "saga-workshop-dtest-9hsr"
dataset_id = "raw"
table_id = "martin-test"

Her benytter vi oss av BigQuery "inline magic" kommandoer for å kjøre en SQL spørring direkte fra notebooken:

In [ ]:
%%bigquery --project $project_id trip_df
SELECT *
FROM `<dataset_id>.<table_id>`

Her benytter vi oss av BigQuery klient biblioteket for å kjøre den samme SQL spørringen:

In [ ]:
client = bigquery.Client(project=project_id)

trip_df = client.query(
    """
    SELECT *
    FROM `%s.%s`
    """ % (dataset_id, table_id)
).to_dataframe()

In [ ]:
trip_df.head()

# Importe data med Dataflow inn i BigQuery

TODO: Tekst her


In [ ]:
!gcloud auth login

In [ ]:
!gcloud projects list

In [ ]:
!gcloud config set project saga-workshop-dtest-9hsr

In [ ]:
!gcloud auth application-default login

In [ ]:
import sys
!{sys.executable} -m pip install 'apache-beam[interactive]'

In [ ]:
import sys
!{sys.executable} -m pip install 'apache-beam[gcp]'

In [ ]:
!gcloud auth login

In [ ]:
!gcloud projects list

In [ ]:
!gcloud config set project saga-workshop-dtest-9hsr

In [ ]:
!gcloud auth application-default login

In [ ]:
import csv

# Test

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.io.gcp.gcsio import GcsIO
import apache_beam.io.fileio
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions


In [ ]:
options = pipeline_options.PipelineOptions()

# Set the pipeline mode to stream the data from Pub/Sub.
options.view_as(pipeline_options.StandardOptions).streaming = False
options.view_as(GoogleCloudOptions).project = "saga-workshop-dtest-9hsr"

p = beam.Pipeline(InteractiveRunner(), options=options)

In [ ]:
file_pattern = "gs://saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest/*"

file_lines = (p
  | "read" >> beam.io.fileio.MatchFiles(file_pattern))

In [ ]:
ib.show(files)

In [ ]:
foboie = (file_lines
  | "transform" >> beam.Map(lambda line: csv.DictReader())

In [ ]:
output = "Hello World"
!echo {output}

### Insert data with code

In [ ]:
project_id = "saga-workshop-dtest-9hsr"
dataset_id = "raw"
table_id = ".".join([project_id, dataset_id, "martin_test_table"])
table_id

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object
client = bigquery.Client(project=project_id)

client.create_table(table_id, exists_ok=True)

In [ ]:
rows_to_insert = [
    {u"full_name": u"Phred Phlyntstone", u"age": 32},
    {u"full_name": u"Wylma Phlyntstone", u"age": 29},
]

errors = client.insert_rows_json(
    table_id, rows_to_insert, row_ids=[None] * len(rows_to_insert)
)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
)

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)